In [4]:
import pandas as pd
import spotipy
from sqlalchemy import create_engine
from secrets import spotify_secrets, localhost_db, postgres_db
from spotipy.oauth2 import SpotifyOAuth
from queries import *

In [6]:
DATABASE_URL = localhost_db

def get_top_artists(user_id):
    engine = create_engine(DATABASE_URL)
    df = pd.read_sql_query(top_artists_query, engine, params={'user_id': user_id})
    engine.dispose()
    return df

def get_top_tracks(user_id):
    engine = create_engine(DATABASE_URL)
    df = pd.read_sql_query(top_tracks_query, engine, params={'user_id': user_id})
    engine.dispose()
    return df

def get_top_genres(user_id):
    engine = create_engine(DATABASE_URL)
    df = pd.read_sql_query(top_genres_query, engine, params={'user_id': user_id})
    engine.dispose()
    return df

In [38]:
df_a = get_top_artists('12120382831')
df_a.head()

,rank,artist_id,timeframe,artist,genres,artist_url,artist_image,popularity
0,1,5K4W6rqBFWDnAN6FQUkS6x,Long,Kanye West,chicago rap; rap,https://open.spotify.com/artist/5K4W6rqBFWDnAN...,https://i.scdn.co/image/bd1c6fdf3705cf9b7d0c8a...,91
1,2,1Xyo4u8uXC1ZmMpatF05PJ,Long,The Weeknd,canadian contemporary r&b; canadian pop; pop,https://open.spotify.com/artist/1Xyo4u8uXC1ZmM...,https://i.scdn.co/image/d9a875c37277c35b94c60c...,95
2,3,2elBjNSdBE2Y3f0j1mjrql,Long,Jay Chou,c-pop; mandopop; taiwan pop; zhongguo feng,https://open.spotify.com/artist/2elBjNSdBE2Y3f...,https://i.scdn.co/image/0e5952203ea0140aa5687f...,74
3,4,73sIBHcqh3Z3NyqHKZ7FOL,Long,Childish Gambino,atl hip hop; hip hop; pop rap; rap,https://open.spotify.com/artist/73sIBHcqh3Z3Ny...,https://i.scdn.co/image/2e1d974c14fb66eb5050bf...,80
4,5,50co4Is1HCEo8bhOyUWKpn,Long,Young Thug,atl hip hop; atl trap; gangster rap; hip hop; ...,https://open.spotify.com/artist/50co4Is1HCEo8b...,https://i.scdn.co/image/69ef089edd03d6d5455ed6...,90


In [8]:
df_g = get_top_genres('12120382831')
df_g.head()

,user_id,rank,genre,points,timeframe
0,12120382831,1,Pop,9.344033,Short
1,12120382831,2,Rap,3.595004,Short
2,12120382831,3,Edm,3.471103,Short
3,12120382831,4,Pop Dance,3.471103,Short
4,12120382831,5,Hip Hop,2.998917,Short


In [106]:
def genre_cloud_data(df_a, df_g):
    top_dict = {}
    for tf in ['Short', 'Medium', 'Long']:
        df = df_g.loc[df_g['timeframe'] == tf].drop_duplicates(subset=['points'])[:10]
        df_a2 = df_a.loc[df_a['timeframe'] == tf]
        artists_genres =  df_a2['genres'].str.split('; ').tolist()
        dict_list = []
        weight = 10
        col1 = list(range(5))
        col2 = list(range(5))
        random.shuffle(col1)
        random.shuffle(col2)
        if col1[-1] == col2[0]:
            col2.insert(0, col2.pop())
        colours = col1 + col2
        for _, row in df.iterrows():
            artists = []
            for i, artist_genres in enumerate(artists_genres):
                if row['genre'].lower() in artist_genres:
                    artists.append(df_a2.iloc[i].to_dict())
                if len(artists) >= 5:
                    break
            dict_list.append({
                'genre': row['genre'],
                'points': row['points'],
                'weight': weight,
                'artists': artists,
                'colour': colours[weight-1]
            })
            weight -= 1
        random.shuffle(dict_list)
        top_dict[tf] = dict_list
    return top_dict

In [105]:
df_a.loc[df_a['timeframe'] == 'Short', 'genres'].str.split('; ').tolist()

[['art pop', 'pop'],
 ['modern alternative rock', 'modern rock', 'nu gaze', 'pop', 'rock'],
 ['alternative r&b', 'viral pop'],
 ['canadian contemporary r&b', 'canadian pop', 'pop'],
 ['alternative r&b', 'hip hop', 'lgbtq+ hip hop', 'neo soul', 'pop'],
 ['downtempo', 'dream pop', 'indietronica'],
 ['chicago rap', 'rap'],
 ['dance pop',
  'edm',
  'electro house',
  'house',
  'pop',
  'pop dance',
  'progressive house',
  'uk dance'],
 ['irish pop', 'modern rock', 'pop', 'pop rock', 'shiver pop'],
 ['pop rap', 'rap', 'trap'],
 ['hip hop', 'rap'],
 ['ambient pop', 'dream pop', 'el paso indie', 'shoegaze'],
 ['anime score', 'japanese soundtrack', 'otacore'],
 ['indie pop', 'indie rock', 'shoegaze', 'toronto indie'],
 ['electropop', 'indie pop', 'indie poptimism', 'modern rock', 'pop'],
 ['atl hip hop', 'hip hop', 'pop rap', 'rap'],
 ['east coast hip hop', 'hip hop', 'rap', 'trap'],
 ['dance pop', 'edm', 'electropop', 'pop', 'pop dance', 'tropical house'],
 ['c-pop', 'mandopop', 'taiwan po

In [107]:
df = genre_cloud_data(df_a, df_g)
df

{'Short': [{'genre': 'Rap',
   'points': 3.595003836650588,
   'weight': 9,
   'artists': [{'rank': 7,
     'artist_id': '5K4W6rqBFWDnAN6FQUkS6x',
     'timeframe': 'Short',
     'artist': 'Kanye West',
     'genres': 'chicago rap; rap',
     'artist_url': 'https://open.spotify.com/artist/5K4W6rqBFWDnAN6FQUkS6x',
     'artist_image': 'https://i.scdn.co/image/bd1c6fdf3705cf9b7d0c8ac8e7bbed98e31a1559',
     'popularity': 91},
    {'rank': 10,
     'artist_id': '6MPCFvOQv5cIGfw3jODMF0',
     'timeframe': 'Short',
     'artist': 'Internet Money',
     'genres': 'pop rap; rap; trap',
     'artist_url': 'https://open.spotify.com/artist/6MPCFvOQv5cIGfw3jODMF0',
     'artist_image': 'https://i.scdn.co/image/46ebec0de8627170636e59876af3ac4a72f94216',
     'popularity': 85},
    {'rank': 11,
     'artist_id': '4V8LLVI7PbaPR0K2TGSxFF',
     'timeframe': 'Short',
     'artist': 'Tyler, The Creator',
     'genres': 'hip hop; rap',
     'artist_url': 'https://open.spotify.com/artist/4V8LLVI7PbaPR0K2

In [104]:
for g in df['Short']:
    print(g['genre'])
    for a in g['artists']:
        print(a['artist'], a['genres'])
    print()

Tropical House
Kygo edm; pop; pop dance; tropical house
Anderson .Paak escape room
Eminem detroit hip hop; hip hop; rap
Travis Scott rap
Rae Sremmurd hip hop; melodic rap; mississippi hip hop; pop rap; rap; southern hip hop; trap

Pop
Kanye West chicago rap; rap
The Weeknd canadian contemporary r&b; canadian pop; pop
Childish Gambino atl hip hop; hip hop; pop rap; rap
Young Thug atl hip hop; atl trap; gangster rap; hip hop; melodic rap; pop rap; rap; trap
A$AP Rocky east coast hip hop; hip hop; rap; trap

Rap
Drake canadian hip hop; canadian pop; hip hop; pop rap; rap; toronto rap
BROCKHAMPTON boy band; hip hop; rap
Lana Del Rey art pop; pop
Vince Staples conscious hip hop; escape room; hip hop; rap; underground hip hop
The 1975 modern alternative rock; modern rock; nu gaze; pop; rock

Modern Rock
The Weeknd canadian contemporary r&b; canadian pop; pop
ILLENIUM edm; electropop; melodic dubstep; pop; pop dance; tropical house
Kendrick Lamar conscious hip hop; hip hop; rap; west coast ra

[{'genre': 'Pop',
  'points': 9.344033200851358,
  'weight': 10,
  'artists': [{'rank': 2,
    'artist_id': '1Xyo4u8uXC1ZmMpatF05PJ',
    'timeframe': 'Long',
    'artist': 'The Weeknd',
    'genres': 'canadian contemporary r&b; canadian pop; pop',
    'artist_url': 'https://open.spotify.com/artist/1Xyo4u8uXC1ZmMpatF05PJ',
    'artist_image': 'https://i.scdn.co/image/d9a875c37277c35b94c60c00d2cd256db098505d',
    'popularity': 95},
   {'rank': 6,
    'artist_id': '69GGBxA162lTqCwzJG5jLp',
    'timeframe': 'Long',
    'artist': 'The Chainsmokers',
    'genres': 'dance pop; edm; electropop; pop; pop dance; tropical house',
    'artist_url': 'https://open.spotify.com/artist/69GGBxA162lTqCwzJG5jLp',
    'artist_image': 'https://i.scdn.co/image/960547a625bc2eb742bb3dd170cbc049d2e94cf9',
    'popularity': 85},
   {'rank': 9,
    'artist_id': '45eNHdiiabvmbp4erw26rg',
    'timeframe': 'Long',
    'artist': 'ILLENIUM',
    'genres': 'edm; electropop; melodic dubstep; pop; pop dance; tropical h

In [41]:
artist_genres = df_a['genres'].str.split('; ').tolist()
for i, ag in enumerate(artist_genres):
    print(ag)
    if 'rap' in ag:
        print(df_a.iloc[i].to_dict())

['chicago rap', 'rap']
{'rank': 1, 'artist_id': '5K4W6rqBFWDnAN6FQUkS6x', 'timeframe': 'Long', 'artist': 'Kanye West', 'genres': 'chicago rap; rap', 'artist_url': 'https://open.spotify.com/artist/5K4W6rqBFWDnAN6FQUkS6x', 'artist_image': 'https://i.scdn.co/image/bd1c6fdf3705cf9b7d0c8ac8e7bbed98e31a1559', 'popularity': 91}
['canadian contemporary r&b', 'canadian pop', 'pop']
['c-pop', 'mandopop', 'taiwan pop', 'zhongguo feng']
['atl hip hop', 'hip hop', 'pop rap', 'rap']
{'rank': 4, 'artist_id': '73sIBHcqh3Z3NyqHKZ7FOL', 'timeframe': 'Long', 'artist': 'Childish Gambino', 'genres': 'atl hip hop; hip hop; pop rap; rap', 'artist_url': 'https://open.spotify.com/artist/73sIBHcqh3Z3NyqHKZ7FOL', 'artist_image': 'https://i.scdn.co/image/2e1d974c14fb66eb5050bf3526c84c4f6e2f47ae', 'popularity': 80}
['atl hip hop', 'atl trap', 'gangster rap', 'hip hop', 'melodic rap', 'pop rap', 'rap', 'trap']
{'rank': 5, 'artist_id': '50co4Is1HCEo8bhOyUWKpn', 'timeframe': 'Long', 'artist': 'Young Thug', 'genres': 

In [76]:
initial = 7.5
increment = 0.25
for i in range(10):
    print('ul.genre-cloud li[data-weight="{}"] {{ --size: {}; }}'.format(i+1, initial + i * increment))
    increment += 0.25

ul.genre-cloud li[data-weight="1"] { --size: 7.5; }
ul.genre-cloud li[data-weight="2"] { --size: 8.0; }
ul.genre-cloud li[data-weight="3"] { --size: 9.0; }
ul.genre-cloud li[data-weight="4"] { --size: 10.5; }
ul.genre-cloud li[data-weight="5"] { --size: 12.5; }
ul.genre-cloud li[data-weight="6"] { --size: 15.0; }
ul.genre-cloud li[data-weight="7"] { --size: 18.0; }
ul.genre-cloud li[data-weight="8"] { --size: 21.5; }
ul.genre-cloud li[data-weight="9"] { --size: 25.5; }
ul.genre-cloud li[data-weight="10"] { --size: 30.0; }


In [97]:
import random

col1 = list(range(5))
col2 = list(range(5))
random.shuffle(col1)
random.shuffle(col2)
if col1[-1] == col2[0]:
    col2.insert(0, col2.pop())
col1 + col2

[3, 4, 2, 1, 0, 4, 0, 1, 2, 3]